Import Packages and Data

In [1]:
import pandas as pd
import datetime as dt
import numpy as np
import seaborn as sns
import os
main = pd.read_excel('Report.xlsx')

Clean Data

In [2]:
main = main[['Place', 'Commodity Code', 'Bill ID', 'Billing Period', 'Start Date', 'End Date', 'Native Use', 'Native Use Unit', 'Cost']]

main = main.replace({'Place': {'Prince George\'s Publick Playhouse / Art Center' : 'Publick Playhouse',
                               'Prince George\'s Publick Playhouse': 'Publick Playhouse',
                               'Palmer Park Community Center / Community Center': 'Palmer Park Community Center',
                               'Laurel Beltsville Senior Activity Center/ Center': 'Laurel Beltsville Senior Activity Center',
                               'Southern Tech&Rec Complex/Athletic Center': 'Southern Tech&Rec Complex'}})

# seperate into dataframes by place
facility = {
    'Publick Playhouse': main.loc[main['Place'] == 'Publick Playhouse'],
    'Palmer Park Community Center': main.loc[main['Place'] == 'Palmer Park Community Center'],
    'Southern Area Aquatics and Recreation Complex': main.loc[main['Place'] == 'Southern Area Aquatics and Recreation Complex'],
    'Laurel Beltsville Senior Activity Center': main.loc[main['Place'] == 'Laurel Beltsville Senior Activity Center'],
    'Southern Tech&Rec Complex': main.loc[main['Place'] == 'Southern Tech&Rec Complex']   
}

facility['Publick Playhouse'].head()

,Place,Commodity Code,Bill ID,Billing Period,Start Date,End Date,Native Use,Native Use Unit,Cost
0,Publick Playhouse,ELECTRIC,13985,199407,1994-07-01,1994-07-12,200.0,kWh,30.51
1,Publick Playhouse,ELECTRIC,13986,199407,1994-07-12,1994-08-09,180.0,kWh,28.34
2,Publick Playhouse,ELECTRIC,13987,199408,1994-08-09,1994-09-09,200.0,kWh,30.51
3,Publick Playhouse,ELECTRIC,13988,199409,1994-09-09,1994-10-12,220.0,kWh,32.69
4,Publick Playhouse,ELECTRIC,13989,199410,1994-10-12,1994-11-09,180.0,kWh,20.63


In [3]:
# remove all rows where cost=0 and sort by Start Date
for place in facility:
    facility[place] = facility[place][facility[place]['Cost'] != 0]
    facility[place] = facility[place][~facility[place]['Cost'].isna()]
    facility[place] = facility[place].sort_values(by=['Start Date', 'Commodity Code'])
    
# seperate facilities into dictionaries by utility
for place in facility:
    facility[place] = {
        'ELECTRIC': facility[place].loc[main['Commodity Code'] == 'ELECTRIC'],
        'WATER': facility[place].loc[main['Commodity Code'] == 'WATER'],
        'NATURALGAS': facility[place].loc[main['Commodity Code'] == 'NATURALGAS'],
        'TELEPHONE': facility[place].loc[main['Commodity Code'] == 'TELEPHONE']
    }

Aggregate Duplicates

In [4]:
def agg_dup_bill_periods(df):
    # aggregate rows with duplicate end dates
    x = df.copy()
    aggregation_functions = {'Place': 'first', 'Commodity Code': 'first', 'Bill ID': 'last', 'Start Date': 'min', 'End Date': 'max', 'Native Use': 'sum', 'Native Use Unit': 'first', 'Cost': 'sum'}
    x = x.groupby(x['Billing Period']).aggregate(aggregation_functions).reset_index()
    return(x)

for place in facility:
    for utility in facility[place]:
        facility[place][utility] = agg_dup_bill_periods(facility[place][utility])

Find Missing Billing Periods

In [5]:
def missingData(df):
    x = df.copy()    
    missing = pd.DataFrame(columns=['From', 'To', 'Days'])
    i = 1                           
    while i < x.shape[0]:
        days = x.iloc[i]['Start Date']-x.iloc[i-1]['End Date']
        if days > dt.timedelta(days=2):
            missing.loc[len(missing.index)] = [x.iloc[i-1]['End Date'], x.iloc[i]['Start Date'], days] 
        i = i + 1
    
    return(missing)

Graph Utility Cost per Year 

In [6]:
def yearlyUtilityCost(df):
    try:
        #Format Data
        x = df.copy()
        facility = x.iloc[0]['Place']
        utility = x.iloc[0]['Commodity Code']
        
        x['Start Date'] = pd.DatetimeIndex(x['Start Date']).year.astype(str)
        x = x[['Start Date', 'Cost']]
        x = x.groupby('Start Date').sum()
        #Graph Data
        yearCost = sns.barplot(x=x.index, y=x.Cost, data = x, ci=None)
        yearCost.set_title('{}: {} COST/YEAR'.format(facility, utility))
        yearCost.set_ylabel("COST ($)")
        yearCost.set_xlabel("YEAR")
        for entry in yearCost.xaxis.get_ticklabels():
            entry.set_rotation(60)
        
        # save graph
        yearCost.figure.savefig('Report Results/{}/YEARLY {}.png'.format(facility, utility))
        yearCost.figure.clf()
        
        # save details
        return(x)
    except:
        return('error making graph')

Create Graphs and Information in Folders

In [7]:
# make results folder
if not os.path.isdir('Report Results'):
    os.mkdir('Report Results')
    
for place in facility:
    path = 'Report Results/{}'.format(place)
    # make folder for facility
    if not os.path.isdir(path):
        os.mkdir(path)
        
    for utility in facility[place]:
        # create txt file for graph description
        # YEARLY
        with open('{}/YEARLY {}.txt'.format(path, utility), "w") as text_file:
            print('{}\t(YEARLY {})\n\n'.format(place, utility), file=text_file)
            # save missing details
            missing = missingData(facility[place][utility])
            if not missing.empty:
                print('MISSING DATA:\n{}\n\n'.format(missing), file=text_file)
            # save graph information
            details = yearlyUtilityCost(facility[place][utility])
            print(details, file=text_file)

<Figure size 432x288 with 0 Axes>

In [8]:
# save cleaned dataframe
cleaned = pd.DataFrame()
for place in facility:
    for utility in facility[place]:
        cleaned = cleaned.append(facility[place][utility], ignore_index=True)
cleaned.to_csv('cleaned_report.csv')

In [9]:
# remove rows where utility use = 0
for place in facility:
    for utility in facility[place]:
        x = facility[place][utility]
        x = x[~x['Native Use'].isna()]

In [10]:
def EUI(df, sqft):
    try:
        #Format Data
        x = df.copy()
        facility = x.iloc[0]['Place']
        x.loc[x['Commodity Code'] == "ELECTRIC", "EnergykBTU"] = x['Native Use'] * float(3.412)
        x.loc[x['Commodity Code'] == "NATURALGAS", "EnergykBTU"] = x['Native Use'] * float(99.9761)
        x['Start Date'] = pd.DatetimeIndex(x['Start Date']).year.astype(str)
        x['EUI'] = x['EnergykBTU']/sqft
        x = x[['Start Date', 'EUI']]
        x = x.groupby('Start Date').sum()
        
        #Graph Data
        eui = sns.barplot(x=x.index, y=x.EUI, data = x, ci=None)
        eui.set_title('{}: EUI'.format(facility, utility))
        eui.set_ylabel("EUI")
        eui.set_xlabel("YEAR")
        for entry in eui.xaxis.get_ticklabels():
            entry.set_rotation(60)
        
        # save graph
        eui.figure.savefig('Report Results/{}/EUI.png'.format(facility))
        eui.figure.clf()
        
        # save details
        return(x)
    except:
        return('error making graph')
    
#EUI
facility_area = {'Publick Playhouse': 14190, 
                 'Palmer Park Community Center': 21921, 
                 'Southern Area Aquatics and Recreation Complex': 75984, 
                 'Laurel Beltsville Senior Activity Center': 16925, 
                 'Southern Tech&Rec Complex': 37048}

for place in facility:
    path = 'Report Results/{}'.format(place)
    df_eui = pd.DataFrame()
    df_eui = df_eui.append(facility[place]['ELECTRIC'], ignore_index=True)
    df_eui = df_eui.append(facility[place]['NATURALGAS'], ignore_index=True)

    with open('{}/EUI.txt'.format(path), "w") as text_file:
        print('{}\t(EUI)\n\n'.format(place), file=text_file)
        # save missing details
        missing = missingData(facility[place]['ELECTRIC'])
        if not missing.empty:
            print('MISSING DATA in ELECTRIC:\n{}\n\n'.format(missing), file=text_file)
        missing = missingData(facility[place]['NATURALGAS'])
        if not missing.empty:
            print('MISSING DATA in NATURALGAS:\n{}\n\n'.format(missing), file=text_file)
        # save graph information
        details = EUI(df_eui, facility_area[place])
        print(details, file=text_file)

<Figure size 432x288 with 0 Axes>